<a href="https://colab.research.google.com/github/cevateness/set_partitioning_problem/blob/main/spp_algos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
#!pip install pulp
#!pip install ortools

In [47]:
import random

def is_feasible_solution_set(elements, subsets):
    covered_elements = set()
    for subset in subsets:
        covered_elements.update(subset["set"])
    return covered_elements == elements

def generate_problem_instance(num_elements, num_subsets):
    elements = set(range(1, num_elements + 1))
    while True:
        subsets = []
        for _ in range(num_subsets):
            subset_size = random.randint(1, num_elements // 2)  # Random subset size
            subset_elements = set(random.sample(elements, subset_size))
            cost = random.randint(1, 10)  # Random cost for the subset
            subsets.append({"set": subset_elements, "cost": cost})

        # Check if the generated instance is feasible
        if is_feasible_solution_set(elements, subsets):
            break  # If feasible, break the loop and return the instance

    return elements, subsets

# Example usage
num_elements = 10
num_subsets = 20

elements, subsets = generate_problem_instance(num_elements, num_subsets)

print("Generated Elements:", elements)
print("Generated Subsets:")
for i, subset in enumerate(subsets):
    print(f"Subset {i + 1}: {subset}")


Generated Elements: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Generated Subsets:
Subset 1: {'set': {1, 10}, 'cost': 5}
Subset 2: {'set': {1, 2, 5, 6, 9}, 'cost': 5}
Subset 3: {'set': {9, 1}, 'cost': 5}
Subset 4: {'set': {6}, 'cost': 7}
Subset 5: {'set': {9, 7}, 'cost': 3}
Subset 6: {'set': {8, 1, 10, 5}, 'cost': 7}
Subset 7: {'set': {1}, 'cost': 5}
Subset 8: {'set': {10, 7}, 'cost': 7}
Subset 9: {'set': {10, 3, 4}, 'cost': 4}
Subset 10: {'set': {4, 6}, 'cost': 1}
Subset 11: {'set': {8, 1, 10, 4}, 'cost': 7}
Subset 12: {'set': {8}, 'cost': 1}
Subset 13: {'set': {5, 6}, 'cost': 1}
Subset 14: {'set': {2, 5, 6, 7, 9}, 'cost': 2}
Subset 15: {'set': {9, 5, 1, 6}, 'cost': 7}
Subset 16: {'set': {8}, 'cost': 9}
Subset 17: {'set': {9, 5, 1}, 'cost': 8}
Subset 18: {'set': {9, 4}, 'cost': 8}
Subset 19: {'set': {10, 3, 4}, 'cost': 9}
Subset 20: {'set': {10, 4, 7}, 'cost': 2}


<ipython-input-47-0fa4daab2b13>:15: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  subset_elements = set(random.sample(elements, subset_size))


In [48]:
import itertools
import numpy as np

# Helper function to check if a solution is feasible (covers all elements exactly once)
def is_feasible_solution(solution, elements):
    # Combine all the selected subsets
    covered_elements = set()
    for subset in solution:
        covered_elements.update(subset['set'])

    # Ensure that all elements are covered exactly once
    if covered_elements != elements:
        return False

    # Check that no element is covered more than once
    for elem in elements:
        count = sum([1 for subset in solution if elem in subset['set']])
        if count != 1:
            return False

    return True

# Brute-force approach (exhaustive search) to find the optimal solution
def brute_force_solution():
    min_cost = float('inf')
    best_solution = None
    for r in range(1, len(subsets) + 1):
        for combination in itertools.combinations(subsets, r):
            if is_feasible_solution(combination,elements):
                total_cost = sum(subset['cost'] for subset in combination)
                if total_cost < min_cost:
                    min_cost = total_cost
                    best_solution = combination
    return min_cost, best_solution




In [49]:
# Branch and Bound
def branch_and_bound():
    def bb_recursive(current_solution, remaining_subsets):
        # Base case: if the current solution is feasible
        if is_feasible_solution(current_solution,elements):
            return sum(subset['cost'] for subset in current_solution), current_solution

        # If no remaining subsets are left, return infinity cost
        if not remaining_subsets:
            return float('inf'), None

        # Branching: choose to include or exclude the next subset
        next_subset = remaining_subsets[0]
        cost_with, solution_with = bb_recursive(current_solution + [next_subset], remaining_subsets[1:])
        cost_without, solution_without = bb_recursive(current_solution, remaining_subsets[1:])

        # Return the better solution (with the minimum cost)
        if cost_with < cost_without:
            return cost_with, solution_with
        else:
            return cost_without, solution_without

    return bb_recursive([], subsets)

In [50]:
# Solver-based approach using PuLP for comparison
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

def pulp_solver_based_solution():
    problem = LpProblem("Set_Partitioning", LpMinimize)

    # Decision variables
    x = [LpVariable(f"x_{i}", cat="Binary") for i in range(len(subsets))]

    # Objective function
    problem += lpSum(x[i] * subsets[i]["cost"] for i in range(len(subsets)))

    # Constraints: cover each element exactly once
    for elem in elements:
        problem += lpSum(x[i] for i in range(len(subsets)) if elem in subsets[i]["set"]) == 1

    # Solve the problem
    problem.solve()

    solution = [subsets[i] for i in range(len(subsets)) if x[i].varValue == 1]
    total_cost = sum(subset["cost"] for subset in solution)
    return total_cost, solution



In [51]:
from ortools.sat.python import cp_model

def constraint_programming_solution():
    model = cp_model.CpModel()

    # Decision variables
    x = [model.NewBoolVar(f"x_{i}") for i in range(len(subsets))]

    # Constraints: Cover each element exactly once
    for elem in elements:
        model.Add(sum(x[i] for i in range(len(subsets)) if elem in subsets[i]['set']) == 1)

    # Objective function: Minimize the total cost
    model.Minimize(sum(x[i] * subsets[i]["cost"] for i in range(len(subsets))))

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        solution = [subsets[i] for i in range(len(subsets)) if solver.Value(x[i]) == 1]
        total_cost = sum(subset["cost"] for subset in solution)
        return total_cost, solution
    else:
        return None, None



In [52]:
constraint_programming_solution()

(12,
 [{'set': {1}, 'cost': 5},
  {'set': {3, 4, 10}, 'cost': 4},
  {'set': {8}, 'cost': 1},
  {'set': {2, 5, 6, 7, 9}, 'cost': 2}])

In [53]:
pulp_solver_based_solution()

(12,
 [{'set': {1}, 'cost': 5},
  {'set': {3, 4, 10}, 'cost': 4},
  {'set': {8}, 'cost': 1},
  {'set': {2, 5, 6, 7, 9}, 'cost': 2}])

In [54]:
branch_and_bound()

(12,
 [{'set': {1}, 'cost': 5},
  {'set': {3, 4, 10}, 'cost': 4},
  {'set': {8}, 'cost': 1},
  {'set': {2, 5, 6, 7, 9}, 'cost': 2}])

In [55]:
brute_force_solution()

(12,
 ({'set': {1}, 'cost': 5},
  {'set': {3, 4, 10}, 'cost': 4},
  {'set': {8}, 'cost': 1},
  {'set': {2, 5, 6, 7, 9}, 'cost': 2}))